In [10]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import csv, os, datetime, sys
import ollama

image_dir = "images/"
colpali_output = "colpali_output.csv"
ms_rag_log = "ms_rag_log.csv"
model_name = "qwen_2_VL_2B_Instruct"

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [ ]:
# Get total GPU memory
total_memory = torch.cuda.get_device_properties(0).total_memory

# Get free and total memory
free_memory, total_memory = torch.cuda.mem_get_info()

# Get allocated and reserved memory
allocated_memory = torch.cuda.memory_allocated()
reserved_memory = torch.cuda.memory_reserved()

# Print memory information
print(f"Total memory: {total_memory / (1024**3):.2f} GB")
print(f"Free memory: {free_memory / (1024**3):.2f} GB")
print(f"Allocated memory: {allocated_memory / (1024**3):.2f} GB")
print(f"Reserved memory: {reserved_memory / (1024**3):.2f} GB")

In [7]:
def get_model_processor():
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
    )
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
    return model, processor
    
def generate_response(image, text, prompt, model, processor):
    #generate image output from vlm
    messages = [
        {
            "role": "system",
            "content": "If you do not know the answer from the given image, do not answer."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]

    text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    image_output = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    

    #generate text output from llama
    text_output = ollama.chat(
        model = "llama3.2",
        messages = [
            {
                "role":"system",
                "content": "If you do not know the answer from the given context then do not answer."
            },
            {
                "role": "user",
                "content": f"Context: {text}, Question: {prompt}",
            }]
    )

    #generate aggregated output
    final_output = ollama.chat(
        model = "llama3.2",
        messages = [
            {
                "role":"system",
                "content": "Generate a response based on the image and text context provided to answer the question."
            },
            {
                "role": "user",
                "content": f"Image Context: {image_output}, Text Context: {text_output.message.content}, Question: {prompt}",
            }]
    )
    
    return final_output.message.content
    

def log_response(doc_id, document, page_num, question, response):

    isfile = os.path.isfile(ms_rag_log)
    with open(ms_rag_log, mode='a', newline='') as file:
        writer = csv.writer(file)
    
        if not isfile:
            writer.writerow(["doc_id", "document", "page_num", "question", "response", "model", "datetime"])
        
        writer.writerow([doc_id, document, page_num, question, response, "ms rag", str(datetime.datetime.now())])

def print_colpali_info(row):
    print(f"Question: {row[4]}\nDocument: {row[1]}\nPage Number: {row[2]}")

In [ ]:
model, processor = get_model_processor()

In [5]:
colpali_info = []
with open(colpali_output, mode='r', newline='') as file:
    reader = csv.reader(file)
    
    header = next(reader)

    for row_num, row in enumerate(reader, start=1):
        doc_id, document, page_num, score, question, base64 = row
        colpali_info.append([doc_id, document, page_num, score, question, base64])

In [9]:
colpali_row = 0
print_colpali_info(colpali_info[colpali_row])

Question: What does Figure 1 illustrate in 'A Simple yet Universal Framework for Depth Completion'?
Document: A Simple yet Universal Framework for Depth Completion.pdf
Page Number: 1


In [ ]:
text = "" #I just copy and paste from the pdf document into this string
image = "" #The image corresponding to the page from the above cell will be snipped and saved to the image directory. The filename is entered here
response = generate_response(image_dir + image, text, colpali_info[colpali_row][4], model, processor)
log_response(colpali_info[colpali_row][0], colpali_info[colpali_row][1], colpali_info[colpali_row][2], colpali_info[colpali_row][4], response)